In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке,
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
#
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию,
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными,
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
#
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install scikit-optimize

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.plotting as pd_plt
import pickle

from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RepeatedStratifiedKFold,
    StratifiedShuffleSplit,
    StratifiedKFold,
    KFold,
    cross_val_score,
    HalvingGridSearchCV,
    FixedThresholdClassifier,
)
from sklearn.metrics import (
    classification_report,
    precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_auc_score,
    RocCurveDisplay,
    DetCurveDisplay,
    accuracy_score,
    mean_squared_error,
    fbeta_score,
    make_scorer,
    log_loss,
)
from sklearn.preprocessing import (
    MinMaxScaler,
    StandardScaler,
    RobustScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    KBinsDiscretizer,
    FunctionTransformer,
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_selection import SelectPercentile, RFE

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from sklearn.metrics import precision_recall_curve, fbeta_score

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными,
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать,
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = "train.csv"  # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test = "test.csv"  # содержит только имя файла, без имен папок

In [6]:
# Блок обучения модели (все три модели)

In [7]:
X = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)
y = pd.read_csv("train_Y.csv")

In [8]:
df_train = pd.merge(X, y, left_index=True, right_index=True)

In [9]:
df_train.drop_duplicates(inplace=True)

In [10]:
X_train = df_train.drop("Diabetes_binary", axis=1)
y_train = df_train["Diabetes_binary"]
X_test = df_test

In [11]:
columns_to_remove = [
    "Education",
    "MentHlth",
    "PhysHlth",
    "Stroke",
    "Fruits",
    "Veggies",
    "AnyHealthcare",
    "Smoker",
    "PhysActivity",
    "NoDocbcCost",
]
removing_columns = ColumnTransformer(
    [
        (
            "drop_features",
            "drop",
            columns_to_remove,
        )
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
    n_jobs=-1,
).set_output(transform="pandas")

In [12]:
total_pipeline = Pipeline([("first", removing_columns)])

In [13]:
best_params = {
    "criterion": "entropy",
    "max_features": "sqrt",
    "min_samples_leaf": 5,
    "min_samples_split": 2,
    "n_estimators": 1001,
    "oob_score": True,
    "n_jobs": -1,
}

rf_model = RandomForestClassifier(**best_params)
final_pipeline = Pipeline([("preprocessing", total_pipeline), ("model", rf_model)])

final_pipeline.fit(X_train, y_train)

custom_threshold = 0.281
fin_pipline = FixedThresholdClassifier(final_pipeline, threshold=0.281).fit(
    X_train, y_train
)

In [14]:
best_params_GB = {
    "criterion": "squared_error",
    "learning_rate": 0.01,
    "loss": "exponential",
    "max_depth": 3,
    "max_features": "sqrt",
    "min_samples_leaf": 1,
    "n_estimators": 1318,
}


gb_model = GradientBoostingClassifier(**best_params_GB)
final_pipeline_GB = Pipeline([("preprocessing", total_pipeline), ("model", gb_model)])

final_pipeline_GB.fit(X_train, y_train)

custom_threshold_GB = 0.279
fin_pipline_GB = FixedThresholdClassifier(final_pipeline_GB, threshold=0.279).fit(
    X_train, y_train
)

In [15]:
# Блок предсказания с использованием тестового набора (все три модели)

In [16]:
y_predict_RF = fin_pipline.predict(X_test)

In [17]:
y_predict_GB = fin_pipline_GB.predict(X_test)

Моя финальная модель - бустинг, который я уже выше обучила

In [18]:
y_predict_final = fin_pipline_GB.predict(X_test)

In [19]:
# Вектора предсказанных значений  RandomForestClassifier или ExtraTreesClassifier
y_predict_RF

array([1., 1., 1., ..., 0., 1., 1.])

In [20]:
# Вектора предсказанных значений  GradientBoostingClassifier или HistGradientBoostingClassifier
y_predict_GB

array([1., 1., 1., ..., 0., 1., 1.])

In [21]:
# Вектора предсказанных значений  финальной модели
y_predict_final

array([1., 1., 1., ..., 0., 1., 1.])